# Weekly Report for Hospital Data

In [16]:
# Imports
import sys
import pandas as pd
import psycopg
from loadinghelper import check_numeric_na

from credentials import DB_PASSWORD, DB_USER

In [47]:
# Connect to DB
conn = psycopg.connect(
    host="sculptor.stat.cmu.edu", dbname=DB_USER,
    user=DB_USER, password=DB_PASSWORD
)
cur = conn.cursor()

In [24]:
# Summary I & II
with conn.transaction():
    cur.execute(
        "SELECT report_date AS week, COUNT(*) AS records_loaded, "
        "SUM(total_adult_hospital_beds), SUM(total_pediatric_hospital_beds), "
        "SUM(total_adult_hospital_beds_occupied), "
        "SUM(total_pediatric_hospital_beds_occupied), "
        "SUM(inpatient_beds_occupied_covid) "
        "FROM facility_reports "
        "GROUP BY report_date "
        "ORDER BY week DESC;"
    )

records = pd.DataFrame(
    cur.fetchall(), columns=["Week", "Records Loaded", "Total Adult Hospital Beds Available", 
                             "Total Pediatric Hospital Beds Available", 
                             "Total Adult Hospital Beds In-Use", "Total Pediatric Hospital Beds In-Use",
                             "Total Hospital Beds Used by COVID cases"]
)

In [19]:
# I
records[["Week", "Records Loaded"]]

,Week,Records Loaded
0,2022-09-23,4996


In [20]:
# II
records

,Week,Records Loaded,Total Adult Hospital Beds Available,Total Pediatric Hospital Beds Available,Total Adult Hospital Beds In-Use,Total Pediatric Hospital Beds In-Use,Total Hospital Beds Used by COVID cases
0,2022-09-23,4996,413589.5,39605.5,463698.4,27778.1,25600.6


In [31]:
# III
with conn.transaction():
    cur.execute(
        "SELECT facility_name, rating, "
        "total_adult_hospital_beds_occupied, "
        "total_pediatric_hospital_beds_occupied, "
        "total_icu_beds_occupied, "
        "inpatient_beds_occupied_covid "
        "FROM facility_information info "
        "JOIN quality_ratings rate "
        "ON info.facility_id = rate.facility_id "
        "JOIN facility_reports "
        "ON info.facility_id = hospital_pk "
        "ORDER BY rating DESC;"
        
    )

ratings = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Hospital Rating",
                             "Total Adult Hospital Beds In-Use", "Total Pediatric Hospital Beds In-Use",
                             "Total ICU Beds In-Use", "Total Hospital Beds Used by COVID cases"]
)

In [33]:
ratings

,Hospital Name,Hospital Rating,Total Adult Hospital Beds In-Use,Total Pediatric Hospital Beds In-Use,Total ICU Beds In-Use,Total Hospital Beds Used by COVID cases
0,SETON SMITHVILLE REGIONAL HOSPITAL,None,4.3,0,0,0
1,MADISON COUNTY MEMORIAL HOSPITAL,None,7.4,0,0,0
2,FISHERMEN'S COMMUNITY HOSPITAL,None,None,0,0,0
3,PROFESSIONAL HOSPITAL GUAYNABO INC,None,21.7,0,5,None
4,JOHNS HOPKINS ALL CHILDREN'S HOSPITAL,None,0,192.6,49.7,5.6
...,...,...,...,...,...,...
13765,UNIVERSITY OF ILLINOIS HOSPITAL AND CLINICS,1,325.7,38.7,82.6,18
13766,BERGEN NEW BRIDGE MEDICAL CENTER,1,306.3,10.7,None,None
13767,ROCKCASTLE REGIONAL HOSPITAL & RESPIRATORY CAR...,1,13.9,0,None,None
13768,MONROE COUNTY MEDICAL CENTER,1,17.7,0,0,None


In [71]:
# No data
# Note: Change thisweek to the parameter of this week
with conn.transaction():
    cur.execute(
        "SELECT facility_name, MAX(report_date) "
        "FROM facility_information "
        "JOIN facility_reports ON "
        "facility_id = hospital_pk "
        "GROUP BY facility_id "
        "HAVING MAX(report_date) < %(week)s "
        "ORDER BY MAX(report_date) DESC;",
        {"week": pd.to_datetime("2022-10-21")}
    )

nodata = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Last reported Date"]
)

In [72]:
nodata

,Hospital Name,Last reported Date
0,WAGONER COMMUNITY HOSPITAL,2022-10-14
1,DRUMRIGHT REGIONAL HOSPITAL,2022-10-14
2,KNOX COUNTY HOSPITAL,2022-10-14
3,SYRACUSE AREA HEALTH,2022-10-14
4,UNITY MEDICAL CENTER,2022-10-14
5,WINSTON MEDICAL CENTER & SWINGBED,2022-10-14
6,MASSAC MEMORIAL HOSPITAL,2022-10-14
7,HOLY CROSS HOSPITAL,2022-10-14
8,LA PAZ REGIONAL HOSPITAL,2022-10-14
9,MT SINAI HOSPITAL MEDICAL CENTER,2022-10-14


In [102]:
# State COVID increase
# Note: Change variables!
with conn.transaction():
    cur.execute(
        "SELECT tw.state, tw.covidcase - lw.covidcase AS coviddiff "
        "FROM ("
        # This weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(week)s "
        "GROUP BY state) tw "
        "JOIN ("
        # Last weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(lastweek)s "
        "GROUP BY state) lw "
        "ON tw.state = lw.state "
        "ORDER BY coviddiff DESC;",
        {
            "week": pd.to_datetime("2022-10-21"),
            "lastweek": pd.to_datetime("2022-10-21") - pd.Timedelta(days=7)
        }
    )

statecovid = pd.DataFrame(
    cur.fetchall(), columns=["States", "Change in Covid Case"]
)

In [103]:
statecovid

,States,Change in Covid Case
0,GU,None
1,VI,None
2,IL,133.3
3,WI,81.4
4,OH,79.9
5,IN,67.8
6,MN,65.9
7,AZ,64.3
8,PA,60.4
9,MO,50.1
